In [107]:
import os
import json
import requests
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
from bs4 import BeautifulSoup

## Data Preprocessing & Correction

In [4]:
##### read the files #####
os.chdir('C:/users/Ruibone/Desktop/fubon_stock/')

statement_dict = dict(all = pd.DataFrame())  # read yearly statement (2021 - 2024) into a dictionary
for index, year in enumerate(range(2021, 2025)):
    statement_dict[year] = pd.read_csv(f'statement_{year}.csv', header = 0, encoding = 'big5')
    statement_dict['all'] = pd.concat([statement_dict['all'], statement_dict[year]], axis = 0)
statement_dict['all'] = statement_dict['all'].reset_index(drop = True)


##### restructure the statement dataframe from dictionary #####
keep_column = ['成交日期', '\t交易類別', '\t股票名稱', '\t成交股數', '\t成交單價', '\t成交價金', '\t手續費', '\t交易稅', '\t淨收付金額']
revised_column = ['成交日期', '交易類別', '股票名稱', '成交股數', '成交單價', '成交價金', '手續費', '交易稅', '淨收付金額']

statement_df = statement_dict['all'].copy()[keep_column]
statement_df.columns = revised_column

##### add a row of ESPP (Employee Stock Purchase Plan) at the date of my resignation #####
statement_full = statement_df.copy()
statement_full.loc[len(statement_df)] = ['2023/10/16', '員工認股', '台積電(2330)', 284, 513.07, 145711, 0, 0, -145711]

##### correction for the date format & stock naming #####
statement_done = statement_full.copy()
statement_done['成交日期'] = pd.to_datetime(statement_full['成交日期'])
statement_done = statement_done.replace('復華S＆P500成長(00924)', '復華S&P500成長(00924)')
statement_done.tail()

,成交日期,交易類別,股票名稱,成交股數,成交單價,成交價金,手續費,交易稅,淨收付金額
187,2024-02-05,現股買進,大華優利高填息30(00918),1000,21.38,21380,30.0,0.0,-21410.0
188,2024-02-05,現股賣出,群益台ESG低碳50(00923),3000,17.38,52140,74.0,52.0,52014.0
189,2024-02-15,現股賣出,復華S&P500成長(00924),1000,19.39,19390,27.0,19.0,19344.0
190,2024-02-15,現股買進,富邦NASDAQ正2(00670L),100,109.05,10905,15.0,0.0,-10920.0
191,2023-10-16,員工認股,台積電(2330),284,513.07,145711,0.0,0.0,-145711.0


## Profit Reporter

In [5]:
'''  property
statement -> statement dataframe download from fubon app
startdate / enddate -> time range for statement
all_stock_name -> list of all stocks I used to possess
closing_price -> the closing stock price crawled from TWSE website
unrealized_summary -> summary of unrealized pnl
realized_summary -> summary of realized pnl
'''
# preprocessing fubon statement -> stock price crawler -> realized & unrealized pnl -> diviend crawler -> diviend profit -> total pnl from stock market -> dashboard -> connect to MySQL for data storage
class ProfitReporter:
    
    # input statement dataframe
    def __init__(self, statement):
        self.statement = statement
        self.all_stock_name = self.statement['股票名稱'].unique()


    # select given time interval
    def interval_selector(self, startdate = '2021-01-01', enddate = 'now'):
        self.startdate = startdate 
        if enddate == 'now':  # determine whcih date is the last trading day
            latest_date = datetime.date.today() 
            if latest_date.isoweekday() == 6:
                latest_date -= datetime.timedelta(days = 1)
            elif latest_date.isoweekday() == 7:
                latest_date -= datetime.timedelta(days = 2)
            self.enddate = str(latest_date)
        else:
            self.enddate = enddate
        
        time_filter = [np.all([x >= pd.Timestamp(self.startdate), x <= pd.Timestamp(self.enddate)]) for x in self.statement['成交日期']]
        self.statement = self.statement[time_filter]
        print(f'StartDate: {self.startdate}; EndDate: {self.enddate}')    
        

    # lookup for the stock price at the enddate (download data from Taiwan Stock Exchange Center & Taipei Exchange)
    def price_lookup(self):
        # web crawler from TWSE
        target_date = self.enddate.replace('-', '')
        url_twse = f'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date={target_date}&type=ALL&response=csv'
        req_twse = requests.get(url_twse).text
        price_twse = req_twse.split('\n')

        # get the stock price from twse
        lookup_stockname = [x.split('(')[0] for x in self.all_stock_name] + ['證券名稱']
        lookup_stocksymbol = ['"' + x.split('(')[1][:-1] + '"' for x in self.all_stock_name] + ['證券代號']
        lookup_info = list()
        for row in price_twse:
            if np.any([np.all([x in row, y in row]) for x, y in zip(lookup_stockname, lookup_stocksymbol)]):
                lookup_info.append(row)        

        # data cleansing
        all_price_list = list()
        for row in lookup_info:
            element = row.split('"')[1::2]
            all_price_list.append(element)

        price_df = pd.DataFrame(all_price_list)
        price_df.columns = price_df.iloc[0]
        price_df = price_df.drop(index = 0)   
        price_key =  price_df['證券名稱'] + '(' + price_df['證券代號'] + ')'
        price_value = price_df['收盤價']
        price_dict = dict()
        for key, value in zip(price_key, price_value):
            price_dict[key] = float(value)
        self.closing_price = price_dict

        # web crawler from Taipei Exchange
        t_year, t_month, t_date = str(int(target_date[0:4]) - 1911), target_date[4:6], target_date[6:]
        url_tpex = f'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=htm&d={t_year}/{t_month}/{t_date}&se=EE&s=0,asc,0'
        req_tpex = requests.get(url_tpex).text
        html_tpex = BeautifulSoup(req_tpex, 'html.parser')
        table_tpex = html_tpex.find('table')
        record_tpex = table_tpex.find_all('tr')

        # reconstruct the table
        rows_tpex = list()
        for idx, row in enumerate(record_tpex[1:]):
            temp_row = [x.get_text() for x in row.find_all('td')]
            if idx == 0:
                header = temp_row
            else:
                rows_tpex.append(temp_row)
        price_tpex = pd.DataFrame(rows_tpex, columns = header)   

        # get the stock price in tpex     
        for idx, row in price_tpex.iterrows():
            keyname = str(row['名稱']) + '(' + str(row['代號']) + ')'
            if keyname in self.all_stock_name:
                price_dict[keyname] = row['收盤']
            

    # calculate unrealized profit of each stock & create a unrealized profit table
    def unrealized_calculator(self):
        # sum up all exchange record to find all the unrealized stocks
        state = self.statement.copy()
        state['持有股數'] = (-1)*state['成交股數']*state['淨收付金額']/abs(state['淨收付金額'])
        state['持有股數'] = state['持有股數'].astype('int')
        state_arrange = state.drop(columns = ['成交日期', '交易類別', '成交單價'])
        state_group = state_arrange.groupby(['股票名稱']).sum()
        state_group['收盤價'] =  [self.closing_price[x] for x in state_group.index]
        state_group['未實現總價'] = state_group['持有股數']*state_group['收盤價'].astype('float')

        # calculate average purchase price & pnl
        unrealized_df = state_group[state_group['未實現總價'] > 0].copy()
        avg_price, total_cost = list(), list()
        for idx, row in unrealized_df.iterrows():
            if row['持有股數'] == row['成交股數']:
                temp_avg = (-1)*row['淨收付金額'] / row['持有股數']
                temp_cost = abs(row['淨收付金額'])
            else:
                target_state = self.statement[self.statement['股票名稱'] == idx].copy()
                purchase_state = target_state[target_state['淨收付金額'] < 0]
                temp_avg = (-1)*np.sum(purchase_state['淨收付金額']) / np.sum(purchase_state['成交股數'])
                temp_cost = temp_avg*row['持有股數']
            avg_price.append(temp_avg)
            total_cost.append(temp_cost)
        unrealized_df['成交均價'] = np.round(avg_price, 2)
        unrealized_df['付出成本'] = np.round(total_cost, 0)
        unrealized_df['未實現損益'] = np.round(unrealized_df['未實現總價'] - avg_price*unrealized_df['持有股數'], 0)
        unrealized_df['未實現損益率'] = np.round(unrealized_df['未實現損益'] / unrealized_df['付出成本'] * 100, 2)
        unrealized_df['未實現損益率'] = unrealized_df['未實現損益率'].astype('str') + '%'

        # summarize all unrealized pnl information
        unrealized_summary = dict()
        unrealized_summary['total_invest'] = int(np.sum(avg_price*unrealized_df['持有股數']))
        unrealized_summary['total_value'] = int(np.sum(unrealized_df['未實現總價']))
        unrealized_summary['total_pnl'] = int(np.sum(unrealized_df['未實現損益']))
        unrealized_summary['total_pnl%'] = np.round((unrealized_summary['total_pnl'] / unrealized_summary['total_invest'])*100, 2)
        unrealized_summary['table'] = unrealized_df[['持有股數', '成交均價', '付出成本', '收盤價', '未實現總價', '未實現損益', '未實現損益率']]       
        self.unrealized_summary = unrealized_summary

        print('------------------------------------未實現損益結算------------------------------------')
        print('投資成本: ', unrealized_summary['total_invest'])
        print('投資現值: ', unrealized_summary['total_value'])
        print('未實現總損益: ', unrealized_summary['total_pnl'])
        print('未實現總損益率: ', str(unrealized_summary['total_pnl%']) + '%')
        print('-------------------------------------------------------------------------------------')

     # calculate realized profit of each stock & create 2 realized profit tables
    def realized_calculator(self):
        # calculate average pruchase price for each stock
        avg_price_dict = dict()
        for stock in self.all_stock_name:
            state_df = self.statement.copy()
            target_state = state_df[state_df['股票名稱'] == stock]
            target_purchase = target_state[target_state['淨收付金額'] < 0]
            avg_price = (-1)*np.sum(target_purchase['淨收付金額']) / np.sum(target_purchase['成交股數'])
            avg_price_dict[stock] = avg_price

        # select purchase record from statement and calculate pnl for each records
        state_purchase = state_df[state_df['淨收付金額'] > 0].reset_index(drop = True)
        state_purchase['購入均價'] = [avg_price_dict[x] for x in state_purchase['股票名稱']]
        state_purchase['購入均價'] = state_purchase['購入均價'].astype('float')
        state_purchase['單筆成本'] = np.round(state_purchase['購入均價']*state_purchase['成交股數'], 0)
        state_purchase['單筆損益'] = state_purchase['淨收付金額'] - state_purchase['單筆成本']
        state_purchase['單筆損益率'] = np.round((state_purchase['單筆損益'] / state_purchase['單筆成本'])*100, 2)
        state_purchase['單筆損益率'] = state_purchase['單筆損益率'].astype('str') + '%'
        state_realized = state_purchase[['成交日期', '股票名稱', '成交股數', '成交單價', '淨收付金額', '購入均價', '單筆成本', '單筆損益', '單筆損益率']]

        # group by each stock and calculate pnl
        sub_state = state_df[state_df['淨收付金額'] > 0].reset_index(drop = True)
        sub_purchase = sub_state.drop(columns = ['成交日期', '交易類別']).groupby('股票名稱').sum()
        sub_purchase['購入均價'] = [avg_price_dict[x] for x in sub_purchase.index]
        sub_purchase['購入均價'] = np.round(sub_purchase['購入均價'].astype('float'), 2)
        sub_purchase['購入成本'] = np.round(sub_purchase['購入均價']*sub_purchase['成交股數'], 0)
        sub_purchase['損益'] = sub_purchase['淨收付金額'] - sub_purchase['購入成本']
        sub_purchase['損益率'] = np.round((sub_purchase['損益'] / sub_purchase['購入成本'])*100, 2)
        sub_purchase['損益率'] = sub_purchase['損益率'].astype('str') + '%'
        sub_realized = sub_purchase[['成交股數', '淨收付金額', '購入均價', '購入成本', '損益', '損益率']]

        # summarize all realized pnl information
        realized_summary = dict()
        realized_summary['trading_record'] = state_realized
        realized_summary['group_record'] = sub_realized
        realized_summary['total_invest'] = int(np.sum(sub_realized['購入成本']))
        realized_summary['total_value'] = int(np.sum(sub_realized['淨收付金額']))
        realized_summary['realized_pnl'] = int(np.sum(sub_realized['損益']))
        realized_summary['realized_pnl%'] = np.round((realized_summary['realized_pnl'] / realized_summary['total_invest'])*100, 2)
        self.realized_summary = realized_summary

        print('------------------------------------已實現損益結算------------------------------------')
        print('投資成本: ', realized_summary['total_invest'])
        print('投資報酬: ', realized_summary['total_value'])
        print('已實現總損益: ', realized_summary['realized_pnl'])
        print('已實現總損益率: ', str(realized_summary['realized_pnl%']) + '%')
        print('-------------------------------------------------------------------------------------')


    def diviend_lookup():
        pass
    

    def diviend_calculator():
        pass

In [48]:
# TPEX crawler
url_tpex = 'https://www.twse.com.tw/rwd/zh/exRight/TWT49U?'
payload_tpex = {
    'startDate': profit.startdate.replace('-', ''),
    'endDate': profit.enddate.replace('-', ''),
    '_': 1708569071037
}
res_tpex = requests.post(url_tpex, payload_tpex)

In [116]:
# organize diviend table from TPEX
json_tpex = res_tpex.json()
pd_tpex = pd.DataFrame(json_tpex['data'], columns = json_tpex['fields'])
hist_tpex = pd_tpex[['資料日期', '股票代號', '股票名稱', '除權息前收盤價', '除權息參考價', '權值+息值']]
hist_tpex.columns = ['date', 'stock_code', 'stock', 'before_ex-diviend', 'reference_ex-diviend', 'diviend']
hist_tpex['stock'] = hist_tpex['stock'] + '(' + hist_tpex['stock_code'] + ')'
hist_tpex = hist_tpex.drop(columns = 'stock_code')
date_split = hist_tpex['date'].str.replace('月', '-').str.replace('日', '').str.split('年')
date_combine = [str(int(x[0]) + 1911) + '-' + str(x[1]) for x in date_split]
hist_tpex['date'] = date_combine

# filter out the stocks we have
union_stock = reduce(lambda x, y: x + '|' + y, profit.all_stock_name)
union_stock = union_stock[:-1]
union_stock

C:\Users\Ruibone\AppData\Local\Temp\ipykernel_13628\1436404230.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist_tpex['stock'] = hist_tpex['stock'] + '(' + hist_tpex['stock_code'] + ')'


'元大台灣50(0050)|富邦科技(0052)|富邦台50(006208)|國泰永續高股息(00878)|中信關鍵半導體(00891)|元大MSCI金融(0055)|富邦臺灣中小(00733)|仁寶(2324)|元大台灣高息低波(00713)|復華富時高息低波(00731)|永豐台灣ESG(00888)|群益半導體收益(00927)|富邦公司治理(00692)|群益台ESG低碳50(00923)|凱基優選高股息30(00915)|元大台灣50正2(00631L)|復華S&P500成長(00924)|國泰20年美債正2(00688L)|元大投資級公司債(00720B)|台積電(2330)|大華優利高填息30(00918)|富邦NASDAQ正2(00670L'

In [8]:
profit = ProfitReporter(statement = statement_done)
profit.interval_selector(enddate = '2024-02-21')
profit.price_lookup()
profit.unrealized_calculator()
profit.realized_calculator()

StartDate: 2021-01-01; EndDate: 2024-02-21
------------------------------------未實現損益結算------------------------------------
投資成本:  514898
投資現值:  582908
未實現總損益:  68010
未實現總損益率:  13.21%
-------------------------------------------------------------------------------------
------------------------------------已實現損益結算------------------------------------
投資成本:  424925
投資報酬:  446554
已實現總損益:  21629
已實現總損益率:  5.09%
-------------------------------------------------------------------------------------


In [9]:
profit.unrealized_summary['table']

,持有股數,成交均價,付出成本,收盤價,未實現總價,未實現損益,未實現損益率
股票名稱,,,,,,,
元大台灣50正2(00631L),800,138.55,110843.0,165.2,132160.0,21317.0,19.23%
元大台灣高息低波(00713),2000,47.18,94359.0,51.1,102200.0,7841.0,8.31%
元大投資級公司債(00720B),200,36.70,7340.0,34.84,6968.0,-372.0,-5.07%
凱基優選高股息30(00915),3000,21.73,65189.0,22.54,67620.0,2431.0,3.73%
台積電(2330),150,513.07,76960.0,681.0,102150.0,25190.0,32.73%
國泰20年美債正2(00688L),3000,9.69,29060.0,8.67,26010.0,-3050.0,-10.5%
大華優利高填息30(00918),1000,21.41,21410.0,21.79,21790.0,380.0,1.77%
富邦NASDAQ正2(00670L),100,109.20,10920.0,105.4,10540.0,-380.0,-3.48%
富邦臺灣中小(00733),1000,50.30,50295.0,61.0,61000.0,10705.0,21.28%
